### Field report: PNL and hit rate analysis for 5 basic strategies.
### 17/03/17- v0.1
#### por mrg

In [ ]:
#Load packages
Pkg.add("TimeSeries");Pkg.add("MarketData");Pkg.add("Plots");Pkg.add("PlotlyJS");
Pkg.add("Gadfly");Pkg.add("StatsBase");Pkg.update();

In [98]:
using TimeSeries, MarketData, Plots, PlotlyJS, StatsBase;

INFO: Recompiling stale cache file C:\Users\miguel\.julia\lib\v0.5\StatsBase.ji for module StatsBase.


In [ ]:
typeof(ohlc)

In [ ]:
ohlc

In [ ]:
ohlc["Open"]

In [ ]:
import TimeSeries.moving

In [ ]:
# Función ventana
function moving{T}(ta::TimeArray{T,1}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end])
    @inbounds for i=1:length(vals)
        vals[i] = f(ta.values[i:i+(window-1)])
    end
    padding && (vals = [NaN*ones(window-1); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

function moving{T}(ta::TimeArray{T,2}, f::Function, window::Int; padding::Bool=false)
    tstamps = padding ? ta.timestamp : ta.timestamp[window:end]
    vals    = zeros(ta.values[window:end, :])
    @inbounds for i=1:size(vals,1), j=1:size(vals, 2)
        vals[i, j] = f(ta.values[i:i+(window-1), j])
    end
    padding && (vals = [NaN*ones(ta.values[1:(window-1), :]); vals])
    TimeArray(tstamps, vals, ta.colnames, ta.meta)
end

In [ ]:
#Nice, ya podemos tomar promedios en ventanas de tamaño N = 10
moving(ohlc,mean,10)

In [ ]:
fs = (:mean)

for f in fs
@eval ($f) = moving(ohlc,$f,10)
end

In [ ]:
# Aquí se van a desarrollar las funciones estrategia